In [57]:
import numpy as np
from math import sin, cos, sqrt

In [58]:
class KF:
 
  def __init__(self, n):

    self.n = n
    self.I = np.matrix(np.eye(n))
    self.x = None #state vector
    self.P = None #cov matrix
    self.F = None # state transition matrix
    self.Q = None #noise matrix
  ##################################
  # x' = x + v_x*dt
  # y' = y + v_y*dt
  # vx' = vx
  # vy' = vy
  # u' = [(1,0,dt,0),(0,1,0,dt),(0,0,1,0),(0,0,0,1)]u
  #Therefore F = [(1,0,dt,0),(0,1,0,dt),(0,0,1,0),(0,0,0,1)]
  ##################################
  def start(self, x, P, F, Q):

    self.x = x
    self.P = P
    self.F = F
    self.Q = Q
        
  def setQ(self, Q):
    self.Q = Q

  def updateF(self, dt):
    self.F[0, 2], self.F[1, 3]  = dt, dt
 
  def getx(self):
    return self.x

  def predict(self):
        
    self.x = self.F * self.x
    self.P = self.F * self.P * self.F.T + self.Q
    
  def update(self, z, H, Hx, R):

    y = z - Hx
    PHt = self.P * H.T
    S = H * PHt + R    
    K = PHt * (S.I)
    
    self.x = self.x + K * y
    self.P = (self.I - K * H) * self.P

In [59]:
class FusionEKF:
 
  def __init__(self, d):
    self.initialized = False
    self.timestamp = 0
    self.n = d['number_of_states']
    self.P = d['initial_process_matrix']
    self.F = d['inital_state_transition_matrix']
    self.Q = d['initial_noise_matrix']
    self.radar_R = d['radar_covariance_matrix']
    self.lidar_R = d['lidar_covariance_matrix']
    self.lidar_H = d['lidar_transition_matrix']
    self.a = (d['acceleration_noise_x'], d['acceleration_noise_y'])
    self.kalmanFilter = KF(self.n)

  def updateQ(self, dt):
    
    dt2 = dt * dt
    dt3 = dt * dt2
    dt4 = dt * dt3
    
    x, y = self.a
    
    r11 = dt4 * x / 4
    r13 = dt3 * x / 2
    r22 = dt4 * y / 4
    r24 = dt3 * y /  2
    r31 = dt3 * x / 2 
    r33 = dt2 * x
    r42 = dt3 * y / 2
    r44 = dt2 * y
    
    Q = np.matrix([[r11, 0, r13, 0],
                  [0, r22, 0, r24],
                  [r31, 0, r33, 0], 
                  [0, r42, 0, r44]])
    
    self.kalmanFilter.setQ(Q)
    
  def update(self, data):
    
    dt = time_difference(self.timestamp, data.get_timestamp())
    self.timestamp = data.get_timestamp()
        
    self.kalmanFilter.updateF(dt)
    self.updateQ(dt)
    self.kalmanFilter.predict()
    
    z = np.matrix(data.get_raw()).T
    x = self.kalmanFilter.getx()
    
    if data.get_name() == 'radar':        
      
      px, py, vx, vy = x[0, 0], x[1, 0], x[2, 0], x[3, 0]
      rho, phi, drho = cartesian_to_polar(px, py, vx, vy)
      H = calculate_jacobian(px, py, vx, vy)
      Hx = (np.matrix([[rho, phi, drho]])).T
      R = self.radar_R 
         
    elif data.get_name() == 'lidar':
    
      H = self.lidar_H
      Hx = self.lidar_H * x
      R = self.lidar_R

    self.kalmanFilter.update(z, H, Hx, R)

  def start(self, data):
    
    self.timestamp = data.get_timestamp()
    x = np.matrix([data.get()]).T
    self.kalmanFilter.start(x, self.P, self.F, self.Q)
    self.initialized = True
    
  def process(self, data):
    
    if self.initialized: 
      self.update(data)
    else:
      self.start(data)

  def get(self):
    return self.kalmanFilter.getx()

In [60]:
class DataPoint:
    
  def __init__(self, d):
    self.timestamp = d['timestamp']
    self.name = d['name']
    self.all = d
    self.raw = []
    self.data = []
    
    if self.name == 'state':
      self.data = [d['x'], d['y'], d['vx'], d['vy']]
      self.raw = self.data.copy()
    
    elif self.name == 'lidar':
      self.data = [d['x'], d['y'], 0, 0]
      self.raw = [d['x'], d['y']]
                  
    elif self.name == 'radar':
      x, y, vx, vy = polar_to_cartesian(d['rho'], d['phi'], d['drho'])
      self.data = [x, y, vx, vy]
      self.raw = [d['rho'], d['phi'], d['drho']]
    
    self.all['processed_data'] = self.data
    self.all['raw'] = self.raw
    
  def get_dict(self):
    return self.all

  def get_raw(self):
    return self.raw

  def get(self):
    return self.data

  def get_timestamp(self):
    return self.timestamp

  def get_name(self):
    return self.name

In [61]:
def cartesian_to_polar(x, y, vx, vy, THRESH = 0.0001):
 
  rho = sqrt(x * x + y * y)
  phi = np.arctan2(y, x)
  
  
  if rho < THRESH:
    print("WARNING: in cartesian_to_polar(): d_squared < THRESH")
    rho, phi, drho = 0, 0, 0
  else:
    drho = (x * vx + y * vy) / rho
    
  return rho, phi, drho

def polar_to_cartesian(rho, phi, drho):
  x, y = rho * cos(phi), rho * sin(phi)
  vx, vy = drho * cos(phi) , drho * sin(phi)

  return x, y, vx, vy

def time_difference(t1, t2):
  return (t2 - t1) / 1000000.0


def get_RMSE(predictions, truths):
  
  pxs, pys, vxs, vys = [], [], [], []
  
  for p, t in zip(predictions, truths):

    ppx, ppy, pvx, pvy = p.get()
    tpx, tpy, tvx, tvy = t.get()
    
    pxs += [(ppx - tpx) * (ppx - tpx)]
    pys += [(ppy - tpy) * (ppy - tpy)]
    vxs += [(pvx - tvx) * (pvx - tvx)]
    vys += [(pvy - tvy) * (pvy - tvy)]
    
  px, py = sqrt(np.mean(pxs)), sqrt(np.mean(pys))
  vx, vy = sqrt(np.mean(vxs)), sqrt(np.mean(vys))
  
  return px, py, vx, vy  
  
def calculate_jacobian(px, py, vx, vy, THRESH = 0.0001, ZERO_REPLACEMENT = 0.0001):
    
  d_squared = px * px + py * py 
  d = sqrt(d_squared)
  d_cubed = d_squared * d
  
  if d_squared < THRESH:
 
    print("WARNING: in calculate_jacobian(): d_squared < THRESH")
    H = np.matrix(np.zeros([3, 4]))
 
  else:

    r11 = px / d
    r12 = py / d
    r21 = -py / d_squared
    r22 = px / d_squared
    r31 = py * (vx * py - vy * px) / d_cubed
    r32 = px * (vy * px - vx * py) / d_cubed
  
    H = np.matrix([[r11, r12, 0, 0], 
                  [r21, r22, 0, 0], 
                  [r31, r32, r11, r12]])

  return H

In [62]:
def parse_data(file_path):
 
  all_sensor_data = []
  all_ground_truths = []  

  with open(file_path) as f:
      
    for line in f:
      data = line.split() 
      
      if data[0]  == 'L':
        
        sensor_data = DataPoint({ 
          'timestamp': int(data[3]),
          'name': 'lidar',
          'x': float(data[1]), 
          'y': float(data[2])
        })
        
        g = {'timestamp': int(data[3]),
             'name': 'state',
             'x': float(data[4]),
             'y': float(data[5]),
             'vx': float(data[6]),
             'vy': float(data[7])
        }
          
        ground_truth = DataPoint(g)
                
      elif data[0] == 'R':
          
        sensor_data = DataPoint({ 
          'timestamp': int(data[4]),
          'name': 'radar',
          'rho': float(data[1]), 
          'phi': float(data[2]),
          'drho': float(data[3])
        })
       
        g = {'timestamp': int(data[4]),
             'name': 'state',
             'x': float(data[5]),
             'y': float(data[6]),
             'vx': float(data[7]),
             'vy': float(data[8])
        }
        ground_truth = DataPoint(g)  

      all_sensor_data.append(sensor_data)
      all_ground_truths.append(ground_truth)

  return all_sensor_data, all_ground_truths


def get_state_estimations(EKF, all_sensor_data):
 
  all_state_estimations = []

  for data in all_sensor_data:
    
    EKF.process(data)
  
    x = EKF.get()
    px, py, vx, vy = x[0, 0], x[1, 0], x[2, 0], x[3, 0]

    g = {'timestamp': data.get_timestamp(),
         'name': 'state',
         'x': px,
         'y': py,
         'vx': vx,
         'vy': vy }

    state_estimation = DataPoint(g)  
    all_state_estimations.append(state_estimation)
    
  return all_state_estimations 


In [63]:
def print_EKF_data(all_sensor_data, all_ground_truths, all_state_estimations, RMSE):
 
  px, py, vx, vy = RMSE 

  print("-----------------------------------------------------------")
  print('{:10s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("RMSE:", px, py, vx, vy))
  print("-----------------------------------------------------------")
  print("NUMBER OF DATA POINTS:", len(all_sensor_data))
  print("-----------------------------------------------------------")  

  i = 1
  for s, p, t in zip(all_sensor_data, all_state_estimations, all_ground_truths):
      
    print("-----------------------------------------------------------")
    print("#", i, ":", s.get_timestamp())
    print("-----------------------------------------------------------")  

    if s.get_name() == 'lidar':
      x, y = s.get_raw()
      print('{:15s} | {:8.3f} | {:8.3f} |'.format("LIDAR:", x, y))
    else:
      rho, phi, drho = s.get_raw()
      print('{:15s} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("RADAR:", rho, phi, drho))  

    x, y, vx, vy = p.get()
    print('{:15s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("PREDICTION:", x, y, x, y))  

    x, y, vx, vy = t.get()
    print('{:15s} | {:8.3f} | {:8.3f} | {:8.3f} | {:8.3f} |'.format("TRUTH:", x, y, x, y))  

    i += 1


In [64]:
lidar_R = np.matrix([[0.01, 0], 
                     [0, 0.01]])

radar_R = np.matrix([[0.01, 0, 0], 
                     [0, 1.0e-6, 0], 
                     [0, 0, 0.01]])

lidar_H = np.matrix([[1, 0, 0, 0],
                     [0, 1, 0, 0]])

P = np.matrix([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1000, 0], 
               [0, 0, 0, 1000]])

Q = np.matrix(np.zeros([4, 4]))
F = np.matrix(np.eye(4))

d = {
  'number_of_states': 4, 
  'initial_process_matrix': P,
  'radar_covariance_matrix': radar_R,
  'lidar_covariance_matrix': lidar_R, 
  'lidar_transition_matrix': lidar_H,
  'inital_state_transition_matrix': F,
  'initial_noise_matrix': Q, 
  'acceleration_noise_x': 5, 
  'acceleration_noise_y': 5
}

EKF1 = FusionEKF(d)

In [65]:
all_sensor_data, all_ground_truths = parse_data("/content/probQues.txt")
all_state_estimations = get_state_estimations(EKF1, all_sensor_data)
px, py, vx, vy = get_RMSE(all_state_estimations, all_ground_truths)
print_EKF_data(all_sensor_data, all_ground_truths, all_state_estimations, 
               RMSE = [px, py, vx, vy])

-----------------------------------------------------------
RMSE:      |    5.760 |    5.738 |   12.085 |   22.314 |
-----------------------------------------------------------
NUMBER OF DATA POINTS: 500
-----------------------------------------------------------
-----------------------------------------------------------
# 1 : 1477010443000000
-----------------------------------------------------------
LIDAR:          |    0.312 |    0.580 |
PREDICTION:     |    0.312 |    0.580 |    0.312 |    0.580 |
TRUTH:          |    0.600 |    0.600 |    0.600 |    0.600 |
-----------------------------------------------------------
# 2 : 1477010443050000
-----------------------------------------------------------
RADAR:          |    1.015 |    0.554 |    4.893 |
PREDICTION:     |    0.784 |    0.730 |    0.784 |    0.730 |
TRUTH:          |    0.860 |    0.600 |    0.860 |    0.600 |
-----------------------------------------------------------
# 3 : 1477010443100000
----------------------------